In [1]:
! pip install pyyaml h5py  # Required to save models in HDF5 format

ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)


2.15.0


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 3s 0us/step


In [4]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10

27/32 [========================>.....] - ETA: 0s - loss: 1.3025 - sparse_categorical_accuracy: 0.6204
Epoch 1: saving model to training_1\cp.ckpt
32/32 [==============================] - 3s 29ms/step - loss: 1.2090 - sparse_categorical_accuracy: 0.6490 - val_loss: 0.7326 - val_sparse_categorical_accuracy: 0.7950
Epoch 2/10
27/32 [========================>.....] - ETA: 0s - loss: 0.4290 - sparse_categorical_accuracy: 0.8750
Epoch 2: saving model to training_1\cp.ckpt
32/32 [==============================] - 1s 23ms/step - loss: 0.4237 - sparse_categorical_accuracy: 0.8790 - val_loss: 0.6157 - val_sparse_categorical_accuracy: 0.8090
Epoch 3/10
29/32 [==========================>...] - ETA: 0s - loss: 0.2795 - sparse_categorical_accuracy: 0.9289
Epoch 3: saving model to training_1\cp.ckpt
32/32 [==============================] - 1s 24ms/step - loss: 0.2858 - sparse_categorical_accuracy: 0.9270 - val_loss: 0.4661 - val_sparse_categorical_accuracy: 0.8480
Epoch 4/10
32/32 [======

In [6]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3806 - sparse_categorical_accuracy: 0.1060 - 357ms/epoch - 11ms/step
Untrained model, accuracy: 10.60%


In [7]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4101 - sparse_categorical_accuracy: 0.8640 - 133ms/epoch - 4ms/step
Restored model, accuracy: 86.40%


In [8]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*n_batches)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2\cp-0005.ckpt

Epoch 10: saving model to training_2\cp-0010.ckpt

Epoch 15: saving model to training_2\cp-0015.ckpt

Epoch 20: saving model to training_2\cp-0020.ckpt

Epoch 25: saving model to training_2\cp-0025.ckpt

Epoch 30: saving model to training_2\cp-0030.ckpt

Epoch 35: saving model to training_2\cp-0035.ckpt

Epoch 40: saving model to training_2\cp-0040.ckpt

Epoch 45: saving model to training_2\cp-0045.ckpt

Epoch 50: saving model to training_2\cp-0050.ckpt


In [9]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [10]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4772 - sparse_categorical_accuracy: 0.8780 - 407ms/epoch - 13ms/step
Restored model, accuracy: 87.80%


In [11]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4772 - sparse_categorical_accuracy: 0.8780 - 444ms/epoch - 14ms/step
Restored model, accuracy: 87.80%


In [12]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a `.keras` zip archive.
model.save('my_model.keras')

Epoch 1/5
32/32 [==============================] - 1s 10ms/step - loss: 1.2260 - sparse_categorical_accuracy: 0.6420
Epoch 2/5
32/32 [==============================] - 0s 11ms/step - loss: 0.4485 - sparse_categorical_accuracy: 0.8740
Epoch 3/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2823 - sparse_categorical_accuracy: 0.9340
Epoch 4/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2165 - sparse_categorical_accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 11ms/step - loss: 0.1511 - sparse_categorical_accuracy: 0.9680


In [13]:
new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4064 - sparse_categorical_accuracy: 0.8660 - 389ms/epoch - 12ms/step
Restored model, accuracy: 86.60%
32/32 [==============================] - 0s 3ms/step
(1000, 10)


In [15]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 11ms/step - loss: 1.1379 - sparse_categorical_accuracy: 0.6650
Epoch 2/5
32/32 [==============================] - 0s 11ms/step - loss: 0.4156 - sparse_categorical_accuracy: 0.8860
Epoch 3/5
32/32 [==============================] - 0s 11ms/step - loss: 0.2964 - sparse_categorical_accuracy: 0.9220
Epoch 4/5
32/32 [==============================] - 0s 11ms/step - loss: 0.1980 - sparse_categorical_accuracy: 0.9550
Epoch 5/5
32/32 [==============================] - 0s 11ms/step - loss: 0.1465 - sparse_categorical_accuracy: 0.9740
INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [16]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 1s - loss: 0.4299 - sparse_categorical_accuracy: 0.8580 - 522ms/epoch - 16ms/step
Restored model, accuracy: 85.80%
32/32 [==============================] - 0s 3ms/step
(1000, 10)


In [18]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 2s 10ms/step - loss: 1.1900 - sparse_categorical_accuracy: 0.6620
Epoch 2/5
32/32 [==============================] - 0s 10ms/step - loss: 0.4257 - sparse_categorical_accuracy: 0.8830
Epoch 3/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2859 - sparse_categorical_accuracy: 0.9210
Epoch 4/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2094 - sparse_categorical_accuracy: 0.9470
Epoch 5/5
32/32 [==============================] - 0s 11ms/step - loss: 0.1492 - sparse_categorical_accuracy: 0.9710


c:\Users\Dr. Ismail\Desktop\tf\TensorFlow-Notebooks\tf\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4212 - sparse_categorical_accuracy: 0.8730 - 366ms/epoch - 11ms/step
Restored model, accuracy: 87.30%
